In [2]:
%load_ext cudf.pandas

In [3]:
import numpy as np, pandas as pd
pd.set_option("display.max_columns", 500)

train = pd.read_csv("dataset/train.csv")
train["Policy Start Date"] = pd.to_datetime( train["Policy Start Date"] )
train["year"] = train["Policy Start Date"].dt.year.astype("float32")
train["month"] = train["Policy Start Date"].dt.month.astype("float32")
train["day"] = train["Policy Start Date"].dt.day.astype("float32")
train["seconds"] = (train["Policy Start Date"].astype("int64") // 10**9).astype("float32") 
train["y"] = np.log1p( train["Premium Amount"] )
print(train.shape)
train.head()

(1200000, 26)


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,year,month,day,seconds,y
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0,2023.0,12.0,23.0,1.703345e+09,7.962067
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,<NA>,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0,2023.0,6.0,12.0,1.686583e+09,7.302496
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,<NA>,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0,2023.0,9.0,30.0,1.696087e+09,6.342121
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,<NA>,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0,2024.0,6.0,12.0,1.718206e+09,6.641182
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0,2021.0,12.0,1.0,1.638372e+09,7.612337


In [4]:
test = pd.read_csv("dataset/test.csv")
test["Policy Start Date"] = pd.to_datetime( test["Policy Start Date"] )
test["year"] = test["Policy Start Date"].dt.year.astype("float32")
test["month"] = test["Policy Start Date"].dt.month.astype("float32")
test["day"] = test["Policy Start Date"].dt.day.astype("float32")
test["seconds"] = (test["Policy Start Date"].astype("int64") // 10**9).astype("float32") 
print(test.shape)
test.head()


(800000, 24)


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,year,month,day,seconds
0,1200000,28.0,Female,2310.0,<NA>,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,<NA>,19.0,<NA>,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House,2023.0,6.0,4.0,1.685892e+09
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,<NA>,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment,2024.0,4.0,22.0,1.713799e+09
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,<NA>,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo,2023.0,4.0,5.0,1.680708e+09
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House,2023.0,10.0,25.0,1.698247e+09
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,<NA>,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House,2021.0,11.0,26.0,1.637940e+09


In [7]:
RMV = ["id", "Policy Start Date", "Premium Amount", "y"]
FEATURES = [c for c in train.columns if not c in RMV]
combined = pd.concat([train, test], axis=0, ignore_index=True)

CATS = []
HIGH_CARDINALITY = []
print(f"THE {len(FEATURES)} BASIC FEATURES ARE:")

for c in FEATURES:
  ftype = "numerical"
  if combined[c].dtype == "object":
    CATS.append(c)
    combined[c] = combined[c].fillna("NAN")
    combined[c],_ = combined[c].factorize()
    combined[c] -= combined[c].min()
    ftype = "categorical"
  if combined[c].dtype == "int64":
    combined[c] = combined[c].astype("int32")
  elif combined[c].dtype == "float64":
    combined[c] = combined[c].astype("float32")
    
  n = combined[c].nunique()
  print(f"{c} ({ftype}) with {n} unique values")
  if n >= 9: HIGH_CARDINALITY.append(c)
  
train = combined.iloc[:len(train)].copy()
test = combined.iloc[len(train):].reset_index(drop=True).copy()

print("\nTHE FOLLOWING HAVE 9 OR MORE UNIQUE VALUES:", HIGH_CARDINALITY )

THE 22 BASIC FEATURES ARE:
Age (numerical) with 47 unique values
Gender (numerical) with 2 unique values
Annual Income (numerical) with 97540 unique values
Marital Status (numerical) with 4 unique values
Number of Dependents (numerical) with 5 unique values
Education Level (numerical) with 4 unique values
Occupation (numerical) with 4 unique values
Health Score (numerical) with 640260 unique values
Location (numerical) with 3 unique values
Policy Type (numerical) with 3 unique values
Previous Claims (numerical) with 10 unique values
Vehicle Age (numerical) with 20 unique values
Credit Score (numerical) with 550 unique values
Insurance Duration (numerical) with 9 unique values
Customer Feedback (numerical) with 4 unique values
Smoking Status (numerical) with 2 unique values
Exercise Frequency (numerical) with 4 unique values
Property Type (numerical) with 3 unique values
year (numerical) with 6 unique values
month (numerical) with 12 unique values
day (numerical) with 31 unique values
s

In [ ]:
def target_encode(train, valid, test, col, target="y", kfold=5, smooth=20, agg="mean"):
  
  train["kfold"] = ((train.index) % kfold)
  col_name = "_".join(col)
  train[f"TE_{agg.upper()}" + col_name]= 0.
  for i in range(kfold):
    
    df_tmp = train[train["kfold"] != i]
    if agg == "mean": mn = train[target].mean()
    elif agg == "min": mn = train[target].min()
    elif agg == "max": mn = train[target].max()
    elif agg == "nunique": mn = 0
    df_tmp = df_tmp[col + [target]].groupby(col).agg([agg, "count"]).reset_index()
    
    

In [6]:
from xgboost import XGBRegressor
import xgboost as xgb, time
print(f"Using XGBoost version", xgb.__version__)

Using XGBoost version 2.0.3
